In [1]:
import pandas as pd
import numpy as np
import re

Load Dataset

In [2]:
df = pd.read_csv('../data/US_Accidents_March23_undersample.csv')
df.head(10)

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-5366042,Source1,2,2022-02-09 15:30:35,2022-02-09 17:47:21,42.744212,-73.777739,42.733496,-73.788485,0.920,...,False,False,False,False,False,False,Day,Day,Day,Day
1,A-3989112,Source1,2,2022-03-08 15:06:07,2022-03-08 16:42:56,39.882025,-102.243444,39.878531,-102.246944,0.304,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
2,A-4417542,Source1,2,2022-05-02 06:23:30,2022-05-02 19:25:30,33.866554,-117.998246,33.892132,-118.043715,3.151,...,False,False,False,False,False,False,Day,Day,Day,Day
3,A-4816426,Source1,2,2022-07-02 19:31:34,2022-07-02 20:16:34,39.090377,-94.804167,39.091236,-94.805199,0.081,...,False,False,False,False,False,False,Day,Day,Day,Day
4,A-2256814,Source2,2,2019-01-31 10:05:24,2019-01-31 10:34:50,34.473560,-120.205147,NaN,NaN,0.000,...,False,False,False,False,False,False,Day,Day,Day,Day
5,A-6277713,Source1,2,2021-11-06 00:02:00,2021-11-06 02:07:39,38.911848,-121.026336,38.911086,-121.032444,0.333,...,False,False,False,False,False,False,Night,Night,Night,Night
6,A-4998610,Source1,2,2022-02-10 12:10:00,2022-02-10 13:52:02,37.732748,-120.839599,37.732838,-120.848759,0.501,...,False,False,False,False,True,False,Day,Day,Day,Day
7,A-1814184,Source2,2,2019-11-12 07:16:31,2019-11-12 08:14:42,25.761061,-80.424881,NaN,NaN,0.000,...,False,False,False,False,True,False,Day,Day,Day,Day
8,A-6668224,Source1,2,2021-01-01 00:41:15,2021-01-01 02:17:19,41.094778,-73.934201,41.079498,-73.921901,1.235,...,False,False,False,False,False,False,Night,Night,Night,Night
9,A-4412180,Source1,2,2022-03-08 18:21:00,2022-03-08 19:36:39,28.327212,-81.290318,28.327478,-81.288885,0.089,...,False,False,False,False,False,False,Day,Day,Day,Day


In [3]:
# number of duplicated data
df.duplicated().sum()

0

In [4]:
df.columns

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [5]:
# drop columns that are not useful
not_useful = ['ID', 'Source', 'End_Time', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Weather_Timestamp']
df = df.drop(not_useful, axis=1)
df.columns

Index(['Severity', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Street', 'City',
       'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [6]:
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64'), dtype('bool')],
      dtype=object)

In [7]:
categorical_columns = df.select_dtypes(include=['object', 'bool']).columns
categorical_columns

Index(['Start_Time', 'Street', 'City', 'County', 'State', 'Zipcode', 'Country',
       'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition',
       'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit',
       'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming',
       'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight',
       'Nautical_Twilight', 'Astronomical_Twilight'],
      dtype='object')

In [8]:
numerical_columns = df.select_dtypes(include=['int64', 'int32', 'float64']).columns
numerical_columns

Index(['Severity', 'Start_Lat', 'Start_Lng', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',
       'Precipitation(in)'],
      dtype='object')

In [9]:
for i in categorical_columns:  
  print(i,df[i].unique().size)

Start_Time 235894
Street 58002
City 8883
County 1570
State 49
Zipcode 72354
Country 1
Timezone 4
Airport_Code 1814
Wind_Direction 25
Weather_Condition 96
Amenity 2
Bump 2
Crossing 2
Give_Way 2
Junction 2
No_Exit 2
Railway 2
Roundabout 2
Station 2
Stop 2
Traffic_Calming 2
Traffic_Signal 2
Turning_Loop 1
Sunrise_Sunset 3
Civil_Twilight 3
Nautical_Twilight 3
Astronomical_Twilight 3


In [10]:
# drop country and turning loop because they have only one unique value
df = df.drop(['Country','Turning_Loop'], axis=1)

In [11]:
print("All wind Direction: ", df['Wind_Direction'].unique())

All wind Direction:  ['SSE' 'N' 'SSW' 'NE' 'ENE' 'CALM' 'NNW' 'SE' 'VAR' 'SW' 'WSW' 'NW' 'WNW'
 'NNE' 'E' 'ESE' 'W' 'S' nan 'South' 'West' 'Calm' 'East' 'Variable'
 'North']


In [12]:
# simplify wind directions
direction_mapping = {
    'Calm': 'CALM',
    'West': 'W', 'WSW': 'W', 'WNW': 'W',
    'South': 'S', 'SSW': 'S', 'SSE': 'S',
    'North': 'N', 'NNW': 'N', 'NNE': 'N',
    'East': 'E', 'ESE': 'E', 'ENE': 'E',
    'Variable': 'VAR'
}

# apply the mapping to the 'Wind_Direction' column
df['Wind_Direction'] = df['Wind_Direction'].map(direction_mapping).fillna(df['Wind_Direction'])
print("All wind directions after simplification:", df['Wind_Direction'].unique())

All wind directions after simplification: ['S' 'N' 'NE' 'E' 'CALM' 'SE' 'VAR' 'SW' 'W' 'NW' nan]


In [13]:
# combine unique weather conditions into a single string, separating with '!'
weather_conditions_str = '!'.join(df['Weather_Condition'].dropna().unique().tolist())

# split the string using specified delimiters and filter out repetitive elements
unique_weather_conditions = np.unique(np.array(re.split(
    r"!|\s/\s|\sand\s|\swith\s|Partly\s|Mostly\s|Blowing\s|Freezing\s", weather_conditions_str))).tolist()
print("Weather Conditions:", unique_weather_conditions)

Weather Conditions: ['', 'Clear', 'Cloudy', 'Drifting Snow', 'Drizzle', 'Dust', 'Dust Whirlwinds', 'Fair', 'Fog', 'Hail', 'Haze', 'Heavy Drizzle', 'Heavy Rain', 'Heavy Rain Shower', 'Heavy Sleet', 'Heavy Snow', 'Heavy T-Storm', 'Heavy Thunderstorms', 'Ice Pellets', 'Light ', 'Light Drizzle', 'Light Ice Pellets', 'Light Rain', 'Light Rain Shower', 'Light Rain Showers', 'Light Sleet', 'Light Snow', 'Light Thunderstorms', 'Mist', 'N/A Precipitation', 'Overcast', 'Partial Fog', 'Patches of Fog', 'Rain', 'Rain Showers', 'Sand', 'Scattered Clouds', 'Shallow Fog', 'Showers in the Vicinity', 'Sleet', 'Small Hail', 'Smoke', 'Snow', 'Squalls', 'T-Storm', 'Thunder', 'Thunder in the Vicinity', 'Thunderstorm', 'Thunderstorms', 'Tornado', 'Widespread Dust', 'Windy', 'Wintry Mix']


In [14]:
# map weather conditions to columns
weather_conditions = {
    'Clear': 'Clear',
    'Cloud': 'Cloud|Overcast',
    'Rain': 'Rain|storm',
    'Heavy_Rain': 'Heavy Rain|Rain Shower|Heavy T-Storm|Heavy Thunderstorms',
    'Snow': 'Snow|Sleet|Ice',
    'Heavy_Snow': 'Heavy Snow|Heavy Sleet|Heavy Ice Pellets|Snow Showers|Squalls',
    'Fog': 'Fog'
}

# apply conditions and create new columns
for column, condition in weather_conditions.items():
    df[column] = df['Weather_Condition'].str.contains(condition, case=False, na=False)


In [15]:
weather_conditions = ['Clear', 'Cloud', 'Rain', 'Heavy_Rain', 'Snow', 'Heavy_Snow', 'Fog']

for condition in weather_conditions:
    null_condition = df['Weather_Condition'].isnull()
    df.loc[null_condition, condition] = df.loc[null_condition, 'Weather_Condition']
    df[condition] = df[condition].astype(bool)

df.loc[:, ['Weather_Condition'] + weather_conditions]

df = df.drop('Weather_Condition', axis=1)


/var/folders/dp/ptv4cbpd6fvf95gkd_vy0df00000gn/T/ipykernel_20225/1656484887.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[null_condition, condition] = df.loc[null_condition, 'Weather_Condition']
/var/folders/dp/ptv4cbpd6fvf95gkd_vy0df00000gn/T/ipykernel_20225/1656484887.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[null_condition, condition] = df.loc[null_condition, 'Weather_Condition']
/var/folders/dp/ptv4cbpd6fvf95gkd_vy0df00000gn/T/ipykernel_20225/1656484887.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a fu

In [16]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'])

df['Year'] = df['Start_Time'].dt.year
df['Month'] = df['Start_Time'].dt.month
df['Weekday'] = df['Start_Time'].dt.weekday
df['Hour'] = df['Start_Time'].dt.hour

df.loc[:4,['Start_Time', 'Year', 'Month', 'Weekday', 'Hour']]
df = df.drop('Start_Time', axis=1)

In [17]:
missing_values = pd.DataFrame(df.isnull().sum(), columns=['Missing_Count']).reset_index()
missing_values.columns = ['Feature', 'Missing_Percent(%)']
missing_values['Missing_Percent(%)'] = missing_values['Missing_Percent(%)'] / df.shape[0] * 100
missing_values = missing_values[missing_values['Missing_Percent(%)'] > 0]
missing_values


,Feature,Missing_Percent(%)
3,Street,0.222222
4,City,0.004334
10,Temperature(F),0.544523
11,Wind_Chill(F),5.217100
12,Humidity(%),0.722616
13,Pressure(in),0.198188
14,Visibility(mi),0.626872
15,Wind_Direction,0.982270
16,Wind_Speed(mph),1.561860
30,Sunrise_Sunset,0.561072


In [18]:
# dropping wind chill because it will be hard to impute since it is missing 25% of the data
df = df.drop('Wind_Chill(F)', axis=1)

In [19]:
df = df.dropna(subset=['Street','City','Zipcode','Airport_Code',
                       'Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight', 'Wind_Direction', 'Precipitation(in)'])

In [20]:
# impute missing values with median
for column in ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']:
    df[column] = df[column].fillna(df[column].median())

In [21]:
df.isnull().sum().sum()

0

In [22]:
df.columns

Index(['Severity', 'Start_Lat', 'Start_Lng', 'Street', 'City', 'County',
       'State', 'Zipcode', 'Timezone', 'Airport_Code', 'Temperature(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Sunrise_Sunset',
       'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Clear',
       'Cloud', 'Rain', 'Heavy_Rain', 'Snow', 'Heavy_Snow', 'Fog', 'Year',
       'Month', 'Weekday', 'Hour'],
      dtype='object')

In [24]:
# save the cleaned data
df.to_csv('../data/US_Accidents_March23_cleaned.csv', index=False)

# random sample of the cleaned data
df.sample(n=10000, random_state=42).to_csv('../data/US_Accidents_March23_cleaned_10000rows.csv', index=False)
df.sample(n=100, random_state=42).to_csv('../data/US_Accidents_March23_cleaned_100rows.csv', index=False)